# Final Project: Generating Product Images from Customer Reviews
## CMU 94-844 Generative AI Lab (Fall 2025)


---

# 📋 Quick Start Guide

## How to Use This Notebook:

### For Each Product:
1. **STEP 1:** Go to product website → Copy ALL review text → Paste into a `.txt` file
2. **STEP 2-4:** Run cells to define functions (only need to run once)
3. **STEP 5+:** Run the processing cell to convert `.txt` → `.csv`

### Workflow:
```
Website Reviews → Copy/Paste → .txt file → Python Parser → Clean CSV files
```

### Files You'll Create:
- `amazon_jeans_dump_data.txt` (raw text from website)
- `walmart_chicken_dump_data.txt` (raw text from website)
- `data/baggy_jeans_reviews.csv` (clean structured data)
- `data/chicken_drumsticks_reviews.csv` (clean structured data)

---

## Setup - Run This First

In [1]:
import pandas as pd
import re
from datetime import datetime
import os
import json

# Create directory structure
os.makedirs('user_input', exist_ok=True)
os.makedirs('parsed_data', exist_ok=True)
print("✅ Ready!")

✅ Ready!


---
# Q1: Product Selection and Customer Review Data Collection (5%)

## Selected Products

**Product 1: Baggy Jeans for Woman Men High Waisted Barrel Leg Jeans**
- **Category:** Clothing/Fashion
- **Link:** https://a.co/d/1Rj5tqt
- **Rationale:** Fashion items have rich visual descriptions in reviews about fit, style, color, and texture - perfect for testing AI's ability to extract visual cues from text.

**Product 2: Perdue Fresh Chicken Drumsticks**
- **Category:** Food/Grocery  
- **Link:** https://www.walmart.com/ip/Perdue-No-Antibiotics-Ever-Fresh-Chicken-Drumsticks-Value-Pack-4-5-5-5-lb-Tray/158751412
- **Rationale:** Food products present unique challenges as reviews focus on quality and freshness rather than direct visual appearance.

**Product 3: [TO BE ADDED]**
- **Category:** [Electronics/Home/Beauty/etc.]
- **Link:** [URL]
- **Rationale:** [To be filled]

---

---
## STEP 1: Manually Copy Review Text from Website

### Instructions for Product 1 (Baggy Jeans - Amazon):

1. **Go to the product page:** https://a.co/d/1Rj5tqt
2. **Scroll to the reviews section**
3. **Copy ALL the text** from the top of the page down through multiple reviews
   - Include: Product title, rating, fabric info, etc.
   - Include: All review text (reviewer names, ratings, dates, review content)
4. **Create a new text file:** `amazon_jeans_dump_data.txt` in this folder
5. **Paste the copied text** into the file
6. **Add a separator line:** Type `===REVIEWS===` on a new line between the product info and the first review
7. **Save the file**

**Example format:**
```
Baggy Jeans for Woman Men High Waisted Barrel Leg Jeans...
4.5 out of 5 stars    91 ratings
Fabric type82%Cotton;18%other
...

===REVIEWS===

Jenni
5.0 out of 5 stars 90's Called — And My Teen Answered
Reviewed in the United States on November 5, 2025
Color: Black1Size: LargeVerified Purchase
Our 17-year-old is fully into the oversized jeans trend...
```

✅ **Once you have created `amazon_jeans_dump_data.txt`, move to STEP 2 below**

---
## STEP 2: Define Parsing Function

This function will read your `.txt` file and extract structured data from the messy text.

**Run this cell to define the parser function:**

In [2]:
def parse_amazon_reviews(file_path):
    """
    Parse Amazon reviews from copied text file
    
    Expected format:
    - Product info at top
    - ===REVIEWS=== separator
    - Reviews with pattern: Reviewer name, rating, date, review text
    """
    print(f"Reading file: {file_path}")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Extract product info (before ===REVIEWS===)
    product_info = {}
    
    if '===REVIEWS===' in content:
        header_part, reviews_part = content.split('===REVIEWS===', 1)
        
        # Get product title (first line)
        lines = header_part.strip().split('\n')
        product_info['title'] = lines[0].strip() if lines else ''
        
        # Extract rating if present
        rating_match = re.search(r'(\d+\.\d+)\s+out of 5 stars', header_part)
        if rating_match:
            product_info['avg_rating'] = rating_match.group(1)
        
        # Extract fabric/material info
        if 'Fabric type' in header_part:
            fabric_match = re.search(r'Fabric type(.+)', header_part)
            if fabric_match:
                product_info['fabric'] = fabric_match.group(1).strip()
    else:
        reviews_part = content
    
    # Parse reviews
    reviews = []
    
    # Split by "Reviewed in" to get individual reviews
    review_blocks = re.split(r'Reviewed in', reviews_part)
    
    for block in review_blocks[1:]:  # Skip first empty split
        review = {}
        
        # Extract rating (X.0 out of 5 stars)
        rating_match = re.search(r'(\d+\.\d+)\s+out of 5 stars', block)
        if rating_match:
            review['rating'] = rating_match.group(1)
        else:
            review['rating'] = ''
        
        # Extract date (everything after "Reviewed in" until next line)
        date_match = re.search(r'^\s*(.+?)on\s+([^\n]+)', block)
        if date_match:
            review['location'] = date_match.group(1).strip()
            review['date'] = date_match.group(2).strip()
        else:
            review['location'] = ''
            review['date'] = ''
        
        # Extract review title (text after stars, before "Reviewed in")
        # Look backwards from "Reviewed in" to find the title
        lines = block.split('\n')
        title = ''
        for i, line in enumerate(lines):
            if 'out of 5 stars' in line and i + 1 < len(lines):
                # Title is usually right after the rating line
                title_candidate = line.split('out of 5 stars', 1)[1].strip()
                if title_candidate:
                    title = title_candidate
                    break
        review['title'] = title
        
        # Extract color and size if present (improved regex to avoid capturing adjacent fields)
        color_match = re.search(r'Color:\s*([^S\n]+?)(?:Size:|Verified|\n)', block)
        review['color'] = color_match.group(1).strip() if color_match else ''
        
        size_match = re.search(r'Size:\s*([^V\n]+?)(?:Verified|\n)', block)
        review['size'] = size_match.group(1).strip() if size_match else ''
        
        # Extract verified purchase
        review['verified'] = 'Verified Purchase' in block
        
        # Extract review text (after Verified Purchase or size, before Helpful/Report)
        # Remove metadata lines
        text_block = block
        # Remove everything before the first newline after color/size/verified
        text_block = re.sub(r'^.*?Verified Purchase\s*', '', text_block, flags=re.DOTALL)
        # Remove Helpful/Report at the end and other footer text
        text_block = re.sub(r'\s*Helpful\s*Report.*$', '', text_block, flags=re.DOTALL)
        text_block = re.sub(r'\s*Report\s*$', '', text_block, flags=re.DOTALL)
        text_block = re.sub(r'Customer image.*', '', text_block, flags=re.DOTALL)
        text_block = re.sub(r'One person found this helpful.*', '', text_block, flags=re.DOTALL)
        text_block = re.sub(r'\d+ people found this helpful.*', '', text_block, flags=re.DOTALL)
        text_block = re.sub(r'Translate review to English.*', '', text_block, flags=re.DOTALL)
        # Remove duplicate reviewer names and ratings that appear in the text
        text_block = re.sub(r'\n[A-Z][a-z]+\s+[A-Z].*?\n\d+\.\d+\s+out of 5 stars.*?\n', '\n', text_block, flags=re.DOTALL)
        
        review['text'] = text_block.strip()
        
        # Only add if we have actual review text
        if review['text'] and len(review['text']) > 10:
            reviews.append(review)
    
    print(f"✅ Parsed {len(reviews)} reviews")
    print(f"✅ Product: {product_info.get('title', 'N/A')[:60]}...")
    
    return product_info, reviews

print("✅ Parser function defined")

✅ Parser function defined


---
## STEP 3: Define Product Metadata

**Run this cell to define product information:**

In [3]:
def parse_walmart_reviews(file_path):
    """
    Parse Walmart reviews from copied text file
    
    Format: Handles both single-line and multi-line reviews
    - Date Name - Rating out of 5 stars - Title/Text (may continue on next lines)
    """
    print(f"Reading file: {file_path}")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Extract product info
    product_info = {}
    
    if '===DESCRIPTION===' in content and '===REVIEWS===' in content:
        desc_part = content.split('===DESCRIPTION===')[1].split('===REVIEWS===')[0]
        
        if 'PERDUE' in desc_part or 'Perdue' in desc_part:
            product_info['title'] = 'PERDUE Fresh Chicken Drumsticks'
        
        features = []
        if 'No Antibiotics Ever' in desc_part:
            features.append('No Antibiotics Ever')
        if 'protein' in desc_part:
            protein_match = re.search(r'(\d+g protein)', desc_part)
            if protein_match:
                features.append(protein_match.group(1))
        
        product_info['features'] = ' | '.join(features) if features else ''
        reviews_part = content.split('===REVIEWS===')[1]
    else:
        reviews_part = content
    
    reviews = []
    seen = set()
    
    lines = reviews_part.split('\n')
    i = 0
    
    while i < len(lines):
        line = lines[i].strip()
        
        # Skip empty lines, page markers
        if not line or line.startswith('Page ') or line.startswith('Walmart Chicken') or line.startswith('ig other'):
            i += 1
            continue
        
        # Match: Date Name - Rating out of 5 stars - RestOfText
        match = re.match(r'^([A-Z][a-z]{2}\s+\d{1,2},\s+\d{4})\s+(.+?)\s+-\s+(\d+)\s+out of 5 stars\s+-\s+(.*)$', line)
        
        if match:
            date = match.group(1).strip()
            reviewer = match.group(2).strip()
            rating = match.group(3).strip()
            first_part = match.group(4).strip()
            
            # Check if review continues on next lines
            full_text = first_part
            j = i + 1
            while j < len(lines):
                next_line = lines[j].strip()
                # Stop if next review header, page marker, or empty
                if not next_line or re.match(r'^[A-Z][a-z]{2}\s+\d{1,2},\s+\d{4}', next_line) or next_line.startswith('Page '):
                    break
                full_text += ' ' + next_line
                j += 1
            
            i = j  # Move to next review
            
            # Split title and text
            if len(first_part) < 80:
                title = first_part
                text = full_text
            else:
                parts = full_text.split('. ', 1)
                if len(parts) == 2 and len(parts[0]) < 100:
                    title = parts[0]
                    text = full_text
                else:
                    title = full_text[:60]
                    text = full_text
            
            review = {
                'date': date,
                'reviewer': reviewer,
                'rating': rating,
                'title': title,
                'text': text,
                'location': 'United States',
                'verified': True
            }
            
            # Avoid duplicates
            unique_key = f"{date}_{reviewer}_{rating}_{text[:30]}"
            
            if unique_key not in seen and len(text) > 10:
                seen.add(unique_key)
                reviews.append(review)
        else:
            i += 1
    
    print(f"✅ Parsed {len(reviews)} unique reviews")
    print(f"✅ Product: {product_info.get('title', 'Walmart Product')}")
    
    return product_info, reviews

print("✅ Walmart parser function defined")

✅ Walmart parser function defined


---
## STEP 2B: Define Walmart Parser Function

Walmart reviews have a different format than Amazon, so we need a separate parser.

**Run this cell to define the Walmart parser:**

In [4]:
# Product 1: Baggy Jeans
product_1_info = {
    'product_name': 'baggy_jeans',
    'url': 'https://a.co/d/1Rj5tqt',
    'category': 'Clothing',
    'description': 'High waisted baggy jeans with barrel leg cut, streetwear style. 82% Cotton, 18% other materials.',
    'features': 'High waist | Barrel leg | Loose fit | Streetwear style | Cotton blend'
}

# Product 2: Chicken Drumsticks
product_2_info = {
    'product_name': 'chicken_drumsticks',
    'url': 'https://www.walmart.com/ip/158751412',
    'category': 'Food',
    'description': 'Fresh chicken drumsticks, no antibiotics ever, 4.5-5.5 lb value pack',
    'features': 'No antibiotics | Fresh | Value pack | 4.5-5.5 lb'
}

# Product 3: TBD
product_3_info = {
    'product_name': 'product_3',
    'url': '',
    'category': '',
    'description': '',
    'features': ''
}

print("✅ Product info defined")

✅ Product info defined


---
## STEP 4: Define Save Function

This function saves the parsed data to CSV files.

**Run this cell to define the save function:**

In [5]:
def save_to_csv(product_info, parsed_product_info, reviews_list):
    """
    Save product info and reviews to CSV
    
    Parameters:
    -----------
    product_info : dict
        Manually entered product metadata
    parsed_product_info : dict
        Product info parsed from text file
    reviews_list : list of dict
        Parsed reviews
    """
    product_name = product_info['product_name']
    
    # Merge product info
    full_product_info = {**product_info, **parsed_product_info}
    full_product_info['num_reviews'] = len(reviews_list)
    full_product_info['collection_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    full_product_info['collection_method'] = 'manual_copy_paste'
    
    # Save product info to parsed_data folder
    info_df = pd.DataFrame([full_product_info])
    info_file = f'parsed_data/{product_name}_info.csv'
    info_df.to_csv(info_file, index=False)
    print(f"\n✅ Saved product info: {info_file}")
    
    # Save reviews to parsed_data folder
    if reviews_list:
        reviews_df = pd.DataFrame(reviews_list)
        reviews_df['product_name'] = product_name
        reviews_file = f'parsed_data/{product_name}_reviews.csv'
        reviews_df.to_csv(reviews_file, index=False)
        print(f"✅ Saved reviews: {reviews_file}")
        print(f"📊 Total reviews: {len(reviews_list)}")
        
        # Show preview
        print(f"\n📄 Review preview:")
        print(reviews_df[['rating', 'title', 'date']].head(3).to_string(index=False))
    else:
        print("⚠️  No reviews to save")

print("✅ Save function ready")

✅ Save function ready


---
## STEP 5: Process Product 1 (Baggy Jeans)

### ⚠️ Before running this cell, make sure:
1. ✅ You created `amazon_jeans_dump_data.txt` with the raw review text
2. ✅ You ran all cells above (Setup, STEP 2, STEP 3, STEP 4)

### What this cell does:
1. Reads `amazon_jeans_dump_data.txt`
2. Parses the messy text into structured data
3. Saves to CSV files in `data/` folder:
   - `data/baggy_jeans_info.csv` (product metadata)
   - `data/baggy_jeans_reviews.csv` (all reviews)

**Run this cell now:**

In [6]:
# Parse the Amazon jeans reviews
parsed_info, reviews = parse_amazon_reviews('user_input/amazon_jeans_dump_data.txt')

# Save to CSV
save_to_csv(product_1_info, parsed_info, reviews)

print("\n✅ Product 1 (Baggy Jeans) complete!")

Reading file: user_input/amazon_jeans_dump_data.txt
✅ Parsed 10 reviews
✅ Product: Baggy Jeans for Woman Men High Waisted Barrel Leg Jeans Casu...

✅ Saved product info: parsed_data/baggy_jeans_info.csv
✅ Saved reviews: parsed_data/baggy_jeans_reviews.csv
📊 Total reviews: 10

📄 Review preview:
rating                                      title             date
   5.0 Love these Jeans! Super cute and on-trend! November 5, 2025
   5.0                               Really great     June 1, 2025
   4.0                                 Good jeans October 26, 2025

✅ Product 1 (Baggy Jeans) complete!


---
## STEP 6: Process Product 2 (Chicken Drumsticks)

### ⚠️ Before running this cell, make sure:
1. ✅ You created `walmart_chicken_dump_data.txt` with the raw review text
2. ✅ You ran all cells above (Setup, STEP 2, STEP 2B, STEP 3, STEP 4)

### What this cell does:
1. Reads `walmart_chicken_dump_data.txt`
2. Parses the Walmart format into structured data
3. Saves to CSV files in `data/` folder:
   - `data/chicken_drumsticks_info.csv` (product metadata)
   - `data/chicken_drumsticks_reviews.csv` (all reviews)

**Run this cell now:**

In [7]:
# Parse the Walmart chicken reviews using the Walmart parser
parsed_info_2, reviews_2 = parse_walmart_reviews('user_input/walmart_chicken_dump_data.txt')

# Save to CSV
save_to_csv(product_2_info, parsed_info_2, reviews_2)

print("\n✅ Product 2 (Chicken Drumsticks) complete!")

Reading file: user_input/walmart_chicken_dump_data.txt
✅ Parsed 40 unique reviews
✅ Product: PERDUE Fresh Chicken Drumsticks

✅ Saved product info: parsed_data/chicken_drumsticks_info.csv
✅ Saved reviews: parsed_data/chicken_drumsticks_reviews.csv
📊 Total reviews: 40

📄 Review preview:
rating                                                        title         date
     5 We only buy Perdue!!! And only at Walmart! We love our Perdu Aug 18, 2025
     5 refund request: I had an issue with the cheese slices that I Jul 19, 2025
     5 Perdue drumstickas are a must in every fridge! These Perdue  Jul 15, 2025

✅ Product 2 (Chicken Drumsticks) complete!


---
## STEP 7: Prepare Product 3

**TODO:** 
1. Choose a third product from a different category (Electronics/Home/Beauty/etc.)
2. Copy review text to a `.txt` file
3. Update `product_3_info` dictionary above
4. Add code here to process it

In [8]:
# TODO: Add product 3
pass

---
## STEP 8: Verify All Collected Data

**Run this cell to see a summary of all CSV files created:**

In [9]:
import glob

print("📁 Collected Data Files:")
print("="*70)

csv_files = sorted(glob.glob('parsed_data/*.csv'))
if not csv_files:
    print("\n⚠️  No CSV files found. Run Step 5 first.")
else:
    for file in csv_files:
        print(f"\n📄 {file}")
        df = pd.read_csv(file)
        print(f"   Rows: {len(df)}")
        print(f"   Columns: {list(df.columns)}")
        print(f"\n   Preview:")
        print(df.head(2).to_string(index=False))
        print("   ...")

📁 Collected Data Files:

📄 parsed_data/baggy_jeans_info.csv
   Rows: 1
   Columns: ['product_name', 'url', 'category', 'description', 'features', 'title', 'avg_rating', 'fabric', 'num_reviews', 'collection_date', 'collection_method']

   Preview:
product_name                    url category                                                                                      description                                                              features                                                                                               title  avg_rating             fabric  num_reviews     collection_date collection_method
 baggy_jeans https://a.co/d/1Rj5tqt Clothing High waisted baggy jeans with barrel leg cut, streetwear style. 82% Cotton, 18% other materials. High waist | Barrel leg | Loose fit | Streetwear style | Cotton blend Baggy Jeans for Woman Men High Waisted Barrel Leg Jeans Casual Loose Denim Pants Streetwear Clothes         4.5 82%Cotton;18%other           10 202

---
# Q2: Analysis of Customer Reviews with LLM

## Overview
We'll use RAG (Retrieval-Augmented Generation) with semantic chunking to analyze reviews and extract:
1. **Visual Information** - Colors, textures, appearance details
2. **Sentiment Analysis** - Overall sentiment and key themes
3. **Product Features** - Main features mentioned in reviews
4. **Image Generation Prompts** - Detailed prompts for diffusion models

## Output Structure:
- `rag_vector_store/` - FAISS indices and embeddings
- `review_analysis_output/` - Analysis results organized by product

---

In [11]:
# Install required packages (run once)
# !pip install sentence-transformers faiss-cpu openai python-dotenv nltk

import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from dotenv import load_dotenv
import nltk
import os
import pandas as pd
import json

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Create output directories
os.makedirs('rag_vector_store', exist_ok=True)
os.makedirs('review_analysis_output', exist_ok=True)

print("✅ Libraries loaded and directories created")

✅ Libraries loaded and directories created


---
## Q2 STEP 2: Semantic Chunking Functions

Chunk reviews into sentences and create embeddings using sentence transformers.

In [12]:
def chunk_reviews_semantically(reviews_df, text_column='text'):
    """
    Split reviews into sentences for semantic chunking.
    Inspired by HW2 approach.
    """
    from nltk.tokenize import sent_tokenize
    
    sentences_list = []
    
    for idx, row in reviews_df.iterrows():
        # Get review text
        text = row[text_column]
        
        # Tokenize into sentences
        sentences = sent_tokenize(text)
        
        # Store each sentence with metadata
        for sent in sentences:
            if len(sent.strip()) > 10:  # Filter out very short sentences
                sentences_list.append({
                    'review_id': idx,
                    'product': row.get('product_name', 'unknown'),
                    'rating': row.get('rating', ''),
                    'sentence': sent.strip()
                })
    
    sentences_df = pd.DataFrame(sentences_list)
    print(f"✅ Created {len(sentences_df)} sentence chunks from {len(reviews_df)} reviews")
    
    return sentences_df


def create_embeddings(sentences_df, model_name='all-MiniLM-L6-v2'):
    """
    Create embeddings for sentences using SentenceTransformers.
    """
    print(f"Loading embedding model: {model_name}...")
    model = SentenceTransformer(model_name)
    
    sentences = sentences_df['sentence'].tolist()
    
    print(f"Creating embeddings for {len(sentences)} sentences...")
    embeddings = model.encode(sentences, show_progress_bar=True, convert_to_numpy=True)
    
    print(f"✅ Created embeddings with shape: {embeddings.shape}")
    
    return embeddings, model

print("✅ Chunking functions defined")

✅ Chunking functions defined


---
## Q2 STEP 3: Build FAISS Vector Store

Create FAISS index for efficient semantic search.

In [13]:
def build_faiss_index(embeddings):
    """
    Build FAISS index for semantic search.
    Using IndexFlatL2 for exact nearest neighbor search.
    """
    dimension = embeddings.shape[1]
    
    print(f"Building FAISS index with dimension: {dimension}")
    
    # Create index
    index = faiss.IndexFlatL2(dimension)
    
    # Add embeddings to index
    index.add(embeddings.astype('float32'))
    
    print(f"✅ FAISS index built with {index.ntotal} vectors")
    
    return index


def save_vector_store(product_name, index, embeddings, sentences_df):
    """
    Save FAISS index, embeddings, and sentences for later use.
    """
    product_dir = f'rag_vector_store/{product_name}'
    os.makedirs(product_dir, exist_ok=True)
    
    # Save FAISS index
    faiss.write_index(index, f'{product_dir}/faiss_index.bin')
    
    # Save embeddings
    np.save(f'{product_dir}/embeddings.npy', embeddings)
    
    # Save sentences DataFrame
    sentences_df.to_csv(f'{product_dir}/sentences.csv', index=False)
    
    print(f"✅ Saved vector store to {product_dir}/")


def load_vector_store(product_name):
    """
    Load saved vector store.
    """
    product_dir = f'rag_vector_store/{product_name}'
    
    index = faiss.read_index(f'{product_dir}/faiss_index.bin')
    embeddings = np.load(f'{product_dir}/embeddings.npy')
    sentences_df = pd.read_csv(f'{product_dir}/sentences.csv')
    
    print(f"✅ Loaded vector store from {product_dir}/")
    
    return index, embeddings, sentences_df

print("✅ FAISS functions defined")

✅ FAISS functions defined


---
## Q2 STEP 4: RAG Query Function

Retrieve relevant review sentences based on queries.

In [14]:
def rag_retrieve(query, index, sentences_df, embedding_model, k=10):
    """
    Retrieve top-k most relevant sentences for a query using RAG.
    
    Parameters:
    -----------
    query : str
        The search query
    index : faiss.Index
        FAISS index
    sentences_df : pd.DataFrame
        DataFrame containing sentences
    embedding_model : SentenceTransformer
        Model for encoding queries
    k : int
        Number of results to retrieve
    
    Returns:
    --------
    pd.DataFrame : Top-k relevant sentences with metadata
    """
    # Encode query
    query_embedding = embedding_model.encode([query], convert_to_numpy=True).astype('float32')
    
    # Search in FAISS index
    distances, indices = index.search(query_embedding, k)
    
    # Get relevant sentences
    relevant_sentences = sentences_df.iloc[indices[0]].copy()
    relevant_sentences['distance'] = distances[0]
    relevant_sentences['relevance_score'] = 1 / (1 + distances[0])  # Convert distance to similarity
    
    return relevant_sentences


def rag_llm_analysis(query, context_sentences, task_description, model="gpt-4o-mini"):
    """
    Use OpenAI with RAG context to perform analysis.
    
    Parameters:
    -----------
    query : str
        The analysis question
    context_sentences : pd.DataFrame
        Retrieved relevant sentences
    task_description : str
        Description of the task
    model : str
        OpenAI model to use
    
    Returns:
    --------
    str : LLM response
    """
    # Format context from retrieved sentences
    context = "\n".join([
        f"[Rating: {row['rating']}] {row['sentence']}" 
        for _, row in context_sentences.iterrows()
    ])
    
    # Create messages
    messages = [
        {
            "role": "system", 
            "content": f"You are an expert product analyst. Use the provided customer review excerpts to {task_description}."
        },
        {
            "role": "user",
            "content": f"Review excerpts:\n{context}\n\nQuery: {query}"
        }
    ]
    
    # Call OpenAI
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.3
    )
    
    return response.choices[0].message.content

print("✅ RAG query functions defined")

✅ RAG query functions defined


---
## Q2 STEP 5: Process Product Reviews

Create vector stores for both products.

In [15]:
# Load reviews
reviews_jeans = pd.read_csv('parsed_data/baggy_jeans_reviews.csv')
reviews_chicken = pd.read_csv('parsed_data/chicken_drumsticks_reviews.csv')

print(f"Loaded {len(reviews_jeans)} jeans reviews and {len(reviews_chicken)} chicken reviews")

# Process Product 1: Baggy Jeans
print("\n### Processing Product 1: Baggy Jeans ###")
sentences_jeans = chunk_reviews_semantically(reviews_jeans)
embeddings_jeans, model_jeans = create_embeddings(sentences_jeans)
index_jeans = build_faiss_index(embeddings_jeans)
save_vector_store('baggy_jeans', index_jeans, embeddings_jeans, sentences_jeans)

# Process Product 2: Chicken Drumsticks
print("\n### Processing Product 2: Chicken Drumsticks ###")
sentences_chicken = chunk_reviews_semantically(reviews_chicken)
embeddings_chicken, model_chicken = create_embeddings(sentences_chicken)
index_chicken = build_faiss_index(embeddings_chicken)
save_vector_store('chicken_drumsticks', index_chicken, embeddings_chicken, sentences_chicken)

print("\n✅ Vector stores created for both products!")

Loaded 10 jeans reviews and 40 chicken reviews

### Processing Product 1: Baggy Jeans ###
✅ Created 52 sentence chunks from 10 reviews
Loading embedding model: all-MiniLM-L6-v2...
Creating embeddings for 52 sentences...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Created embeddings with shape: (52, 384)
Building FAISS index with dimension: 384
✅ FAISS index built with 52 vectors
✅ Saved vector store to rag_vector_store/baggy_jeans/

### Processing Product 2: Chicken Drumsticks ###
✅ Created 103 sentence chunks from 40 reviews
Loading embedding model: all-MiniLM-L6-v2...
Creating embeddings for 103 sentences...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Created embeddings with shape: (103, 384)
Building FAISS index with dimension: 384
✅ FAISS index built with 103 vectors
✅ Saved vector store to rag_vector_store/chicken_drumsticks/

✅ Vector stores created for both products!


---
## Q2 STEP 6: Analysis Module 1 - Visual Information Extraction

Extract colors, textures, materials, and visual details from reviews.

In [16]:
def extract_visual_information(product_name, index, sentences_df, embedding_model):
    """
    Extract visual information from reviews using RAG.
    """
    print(f"\n### Extracting Visual Information for {product_name} ###")
    
    queries = [
        "What colors are mentioned in the reviews?",
        "What textures and materials are described?",
        "How does the product look and appear visually?",
        "What visual details do customers mention?"
    ]
    
    visual_info = {}
    
    for query in queries:
        print(f"\nQuery: {query}")
        
        # Retrieve relevant sentences
        relevant = rag_retrieve(query, index, sentences_df, embedding_model, k=15)
        
        # Analyze with LLM
        analysis = rag_llm_analysis(
            query=query,
            context_sentences=relevant,
            task_description="extract and summarize visual information from customer reviews"
        )
        
        visual_info[query] = {
            'analysis': analysis,
            'top_sentences': relevant.head(5)[['sentence', 'rating', 'relevance_score']].to_dict('records')
        }
        
        print(f"Analysis: {analysis[:200]}...")
    
    # Save results
    output_dir = f'review_analysis_output/{product_name}'
    os.makedirs(output_dir, exist_ok=True)
    
    with open(f'{output_dir}/visual_information.json', 'w') as f:
        json.dump(visual_info, f, indent=2)
    
    print(f"\n✅ Saved visual information to {output_dir}/visual_information.json")
    
    return visual_info

# Extract for both products
visual_jeans = extract_visual_information('baggy_jeans', index_jeans, sentences_jeans, model_jeans)
visual_chicken = extract_visual_information('chicken_drumsticks', index_chicken, sentences_chicken, model_chicken)


### Extracting Visual Information for baggy_jeans ###

Query: What colors are mentioned in the reviews?
Analysis: The reviews mention the following colors:

1. Dark'ish navy blue
2. Yellow (specifically in reference to the embroidery and stitches)...

Query: What textures and materials are described?
Analysis: The customer reviews describe the jeans as being made from a medium-weight denim that is not stiff, indicating a comfortable texture. One review mentions that the material is "nothing special" and "qu...

Query: How does the product look and appear visually?
Analysis: Based on the customer reviews, the product, which appears to be a pair of jeans, has several notable visual characteristics:

1. **Color and Stitching**: The jeans feature yellow stitching, which is h...

Query: What visual details do customers mention?
Analysis: Customers mention several visual details in their reviews of the jeans:

1. **Quality and Trendiness**: The jeans are described as having great quality an

---
## Q2 STEP 7: Analysis Module 2 - Product Features Extraction

Extract key product features, benefits, and drawbacks mentioned in reviews.

In [17]:
def extract_product_features(product_name, index, sentences_df, embedding_model):
    """
    Extract key product features using RAG.
    """
    print(f"\n### Extracting Product Features for {product_name} ###")
    
    queries = [
        "What are the main features and characteristics of this product?",
        "What do customers like most about this product?",
        "What complaints or issues do customers mention?",
        "What makes this product unique or special?"
    ]
    
    features = {}
    
    for query in queries:
        print(f"\nQuery: {query}")
        
        # Retrieve relevant sentences
        relevant = rag_retrieve(query, index, sentences_df, embedding_model, k=15)
        
        # Analyze with LLM
        analysis = rag_llm_analysis(
            query=query,
            context_sentences=relevant,
            task_description="identify and summarize key product features from customer reviews"
        )
        
        features[query] = {
            'analysis': analysis,
            'top_sentences': relevant.head(5)[['sentence', 'rating', 'relevance_score']].to_dict('records')
        }
        
        print(f"Analysis: {analysis[:200]}...")
    
    # Save results
    output_dir = f'review_analysis_output/{product_name}'
    os.makedirs(output_dir, exist_ok=True)
    
    with open(f'{output_dir}/product_features.json', 'w') as f:
        json.dump(features, f, indent=2)
    
    print(f"\n✅ Saved product features to {output_dir}/product_features.json")
    
    return features

# Extract for both products
features_jeans = extract_product_features('baggy_jeans', index_jeans, sentences_jeans, model_jeans)
features_chicken = extract_product_features('chicken_drumsticks', index_chicken, sentences_chicken, model_chicken)


### Extracting Product Features for baggy_jeans ###

Query: What are the main features and characteristics of this product?
Analysis: Based on the customer review excerpts, the main features and characteristics of the product (likely a pair of jeans) are as follows:

1. **Quality**: Generally, customers appreciate the quality of the...

Query: What do customers like most about this product?
Analysis: Based on the customer review excerpts, the key features that customers appreciate about the product include:

1. **Quality**: Multiple reviews highlight the good quality of the jeans, indicating that ...

Query: What complaints or issues do customers mention?
Analysis: Based on the provided customer review excerpts, the following complaints or issues are mentioned:

1. **Sizing Issues**: One customer reported dissatisfaction with the size they ordered, specifically ...

Query: What makes this product unique or special?
Analysis: Based on the customer review excerpts, the product, likely a

---
## Q2 STEP 8: Analysis Module 3 - Image Generation Prompts

Generate detailed prompts for diffusion models based on review analysis.

In [18]:
def generate_image_prompts(product_name, visual_info, features_info, product_category):
    """
    Generate image generation prompts based on analysis.
    """
    print(f"\n### Generating Image Prompts for {product_name} ###")
    
    # Combine visual and feature information
    context = f"""
Product Category: {product_category}

Visual Information Summary:
{json.dumps({k: v['analysis'] for k, v in visual_info.items()}, indent=2)}

Feature Information Summary:
{json.dumps({k: v['analysis'] for k, v in features_info.items()}, indent=2)}
"""
    
    # Generate prompts using LLM
    prompt_request = """
Based on the customer review analysis above, generate 3 detailed image generation prompts for DALL-E or Stable Diffusion.

Each prompt should:
1. Capture the visual essence of the product from customer descriptions
2. Include specific details about colors, textures, materials, and style
3. Be optimized for text-to-image generation
4. Be 2-3 sentences long

Format your response as:
PROMPT 1: [detailed prompt]
PROMPT 2: [detailed prompt]
PROMPT 3: [detailed prompt]
"""
    
    messages = [
        {"role": "system", "content": "You are an expert at creating detailed prompts for AI image generation models."},
        {"role": "user", "content": context + "\n\n" + prompt_request}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    
    generated_prompts = response.choices[0].message.content
    
    # Parse prompts
    prompts_list = []
    for line in generated_prompts.split('\n'):
        if line.startswith('PROMPT'):
            prompt_text = line.split(':', 1)[1].strip() if ':' in line else line
            prompts_list.append(prompt_text)
    
    # Save results
    output_dir = f'review_analysis_output/{product_name}'
    os.makedirs(output_dir, exist_ok=True)
    
    prompt_data = {
        'product_name': product_name,
        'category': product_category,
        'generated_prompts': prompts_list,
        'full_response': generated_prompts
    }
    
    with open(f'{output_dir}/image_generation_prompts.json', 'w') as f:
        json.dump(prompt_data, f, indent=2)
    
    # Also save as text file for easy reading
    with open(f'{output_dir}/image_generation_prompts.txt', 'w') as f:
        f.write(generated_prompts)
    
    print(f"\n✅ Generated {len(prompts_list)} image prompts")
    print(f"✅ Saved to {output_dir}/image_generation_prompts.json")
    
    print("\n📝 Generated Prompts:")
    for i, prompt in enumerate(prompts_list, 1):
        print(f"\n{i}. {prompt}")
    
    return prompt_data

# Generate for both products
prompts_jeans = generate_image_prompts(
    'baggy_jeans', 
    visual_jeans, 
    features_jeans,
    'Clothing - Baggy Jeans'
)

prompts_chicken = generate_image_prompts(
    'chicken_drumsticks',
    visual_chicken,
    features_chicken,
    'Food - Fresh Chicken'
)


### Generating Image Prompts for baggy_jeans ###

✅ Generated 3 image prompts
✅ Saved to review_analysis_output/baggy_jeans/image_generation_prompts.json

📝 Generated Prompts:

1. Create an image of a pair of high-waisted baggy jeans in a dark'ish navy blue color, featuring trendy yellow stitching that pops against the denim. The jeans should showcase cute metal rivets on the pockets and have a relaxed fit, with big pockets visible on the sides. The overall aesthetic should reflect a hip-hop and cool-girl style, emphasizing comfort and casual fashion.

2. Illustrate a stylish pair of baggy jeans made from medium-weight denim, appearing slightly worn for a casual look. Highlight the distinct yellow embroidery details along the seams and the cute metal rivets on the pockets, while ensuring the jeans have a flattering silhouette. The background should be minimalistic to enhance the focus on the jeans, conveying a trendy, youthful vibe.

3. Design an image featuring a pair of baggy jeans 

---
## Q2 STEP 9: Summary and Verification

Verify all outputs and generate summary.

---
## Q3: Image Generation with Diffusion Models 

**To be implemented after Q2 is complete**

This section will:
- Use prompts from Q2 to generate images
- Test with 2 different models (DALL-E, Stable Diffusion)
- Compare generated images with actual product images

In [19]:
# TODO: Image generation - to be implemented
pass